# Matrix Factorization 모델 구현 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import root_mean_squared_error

np.random.seed(42)

In [12]:
rmse = []
prec_at_k = []
rec_at_k = []
ndcg_value_k = []
n = 1

## 1.Data Loading

 - 데이터 불러오는 load_data() 함수 작성
    - data_indicator =0 : u1~u5으로 나뉘어진 movielens100k data를 통한 cross-validation 진행
    - data_indicator =1 : 동일 user_id를 train,test가 포함할 수 있게 나뉘어진 데이터를 통해 진행

In [2]:
def load_data(base,test,data_indicator=0):
    if data_indicator==0:
        print('Data_indicator =0 Selected\n')
        path = Path('../../../datafile/Recsys/ml-100k')
        train = pd.read_csv(path / base,sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})
        test = pd.read_csv(path / test,sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})

        R_train = pd.pivot_table(train,values='rating',index='user',columns='item')
    
    elif data_indicator==1:
        print('Data_indicator =1 Selected\n')
        path = Path('../../../datafile/Recsys/ml-100k')
        data = pd.read_csv(path / 'u.data',sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})
        
        train = data.groupby('user').sample(frac=0.8,random_state=42)
        train_ind = train.index
        test = data.drop(train_ind)

        print(f'Data Size //  original: {data.shape}, train:{train.shape}, test:{test.shape}')
        R_train = pd.pivot_table(train,values='rating',index='user',columns='item')

    return train,test,R_train

# For Data_Indicator=1

In [ ]:
data_indicator = 1
train,test,R_train = load_data(1,1,data_indicator=1)

Data_indicator =1 Selected

Data Size //  original: (100000, 3), train:(80000, 3), test:(20000, 3)


In [15]:
train.head()

,user,item,rating
3733,1,31,3.0
15932,1,39,4.0
9382,1,163,4.0
17863,1,226,3.0
42456,1,169,5.0


In [16]:
R_train.head()

item,1,2,3,4,5,6,7,8,9,10,...,1670,1671,1672,1673,1674,1675,1677,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,NaN,3.0,NaN,NaN,NaN,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
test.head()

,user,item,rating
1,186,302,3.0
7,253,465,5.0
20,119,392,4.0
21,167,486,4.0
26,38,95,5.0


## 2. model implementation

- MF 모델의 전반적인 구현 및 fit() 메소드를 통한 학습 진행 + Predict() 메소드를 통한 예측 진행

In [6]:
class  MatrixFactorization():
    def __init__(self,k,lr,reg_param,epochs):
        self.k = k
        self.lr = lr
        self.reg_param = reg_param
        self.epochs = epochs
        pass

    def fit(self,R):
        self.R_df = R
        self.n_users,self.n_items = R.shape

        self.obs_rows,self.obs_cols = np.nonzero(R) #R의 observed data에 대한 index 반환
        self.obs_ind = list(zip(self.obs_rows,self.obs_cols))
        
        self.uilist_train = list(zip(R.index,R.columns))

        self.R = np.array(R)    # u,i is index for array
        #P,Q random initialization
        self.P = np.random.random(size=(self.n_users,self.k)) *(6/self.k)
        self.Q = np.random.random(size=(self.n_items,self.k)) *(6/self.k)

        #Bias
        self.b_u = np.zeros(self.n_users)
        self.b_i = np.zeros(self.n_items)

        self.global_mean = R[R>0].mean().mean()
        

        for n in range(self.epochs):
            for u,i in self.obs_ind: #index 기준으로 작동
                    if self.R[u,i] == 0:
                        pass
                    else:
                        e = self.R[u,i] - (np.dot(self.P[u,:],self.Q[i,:].T) + self.b_u[u] + self.b_i[i] +self.global_mean)  #error
                        #user-update
                        self.P[u,:] = self.P[u,:] + self.lr * (e* self.Q[i,:] - self.reg_param*self.P[u,:])
                        #item-update
                        self.Q[i,:] = self.Q[i,:] + self.lr * (e* self.P[u,:] - self.reg_param*self.Q[i,:])

                        #Bias-update
                        self.b_u[u] = self.b_u[u] + self.lr * (e - self.reg_param*self.b_u[u])
                        self.b_i[i] = self.b_i[i] + self.lr * (e - self.reg_param*self.b_i[i])
            if n % 10 == 0:
                R_pred = np.dot(self.P,self.Q.T) + self.b_u[:,np.newaxis] + self.b_i[np.newaxis,:] + self.global_mean
                bias_term =  np.sum(np.square(self.b_u)) + np.sum(np.square(self.b_i))
                # Loss Function
                loss = (np.sum(np.square(self.R - R_pred)) + self.reg_param*(np.sum(np.square(self.P)) + np.sum(np.square(self.Q)) +bias_term)) / len(self.obs_ind)
                print(f'Epoch : {n} , Loss : {loss:4f} , Rooted Loss: {np.sqrt(loss):.2f}')
        return self.P,self.Q,self.b_u,self.b_i
    
    def predict(self,test,exclude_unknowns=True):
        P_df = pd.DataFrame(self.P,index=self.R_df.index)
        Q_df = pd.DataFrame(self.Q,index=self.R_df.columns)
        bu_df = pd.DataFrame(self.b_u,index=self.R_df.index)
        bi_df = pd.DataFrame(self.b_i,index=self.R_df.columns)
        

        if exclude_unknowns == True:
            test_filtered = test[test['user'].isin(self.uilist_train[0]) & (test['item'].isin(self.uilist_train[1]))]
            uilist_test = list(zip(test_filtered['user'],test_filtered['item']))
            prediction = test_filtered.copy()
            pred = []
            for val in uilist_test:
                pred.append((np.dot(P_df.loc[val[0]],Q_df.loc[val[1]].T) + bu_df.loc[val[0]] + bi_df.loc[val[1]])) # P,Q is array
            prediction['rating'] = pred
        
        else:
            """
            Include Unknown (users / items) not shown in train dataset.
            If user nor item is not in train dataset --> 0 => Rating will be calculated as 0 (Nan)
            One component (item or user) not in the test dataset --> 1 ==> Rating is calculated as the average of P or Q vector.
            """
            uilist_test = list(zip(test['user'],test['item']))
    
            prediction = test.copy()
            pred=[]
            for val in uilist_test:
                if (val[0] not in self.uilist_train[0]) & (val[1]not in self.uilist_train[1]):
                    P_inner_product = np.full(self.k,0)
                    Q_inner_product = np.full(self.k,0)
                    bias_u_product = 0
                    bias_i_product = 0
                elif val[0] not in self.uilist_train[0]:
                    P_inner_product = np.full(self.k,1)
                    Q_inner_product = Q_df.loc[val[1]].T
                    bias_u_product = 0
                    bias_i_product = int(bi_df.loc[val[1]])
                elif val[1] not in self.uilist_train[1]:
                    P_inner_product = P_df.loc[val[0]]
                    Q_inner_product = np.full(self.k,1)
                    bias_u_product = int(bu_df.loc[val[0]])
                    bias_i_product = 0
                else:
                    P_inner_product = P_df.loc[val[0]]
                    Q_inner_product = Q_df.loc[val[1]].T
                    bias_u_product = int(bu_df.loc[val[0]])
                    bias_i_product = int(bi_df.loc[val[1]])

                pred.append((np.dot(P_inner_product,Q_inner_product)+ self.global_mean + bias_u_product + bias_i_product))
            prediction['rating'] = pred
            test_filtered = test

        # for val in uilist_test:
        #     pred.append(np.dot(self.P[val[0]],self.Q[val[1]].T))
        
        
        #pred.append(self.R_pred[uilist_test])
        #pred = [test_filtered['user'],test_filtered['item'],pred]

        #else:
        #test_filtered = test[test['user'].isin(self.uilist_train[0]) & test['item'].isin(self.uilist_train[1])]
        #uilist_test = list(zip(test_filtered['user'],test_filtered['item']))
        #pred.append(self.R_pred[uilist_test])
        return prediction,test_filtered

## 3. model training
- 구현한 fit() method를 통해서 데이터 Training, Loss값을 통해서 오차 관측

In [27]:
R_train = R_train.fillna(0) #결측치 채우기

#파라미터
k = 10
lr = 0.0001
reg_param = 0.02
epochs = 50

#모델 호출
mf_model  = MatrixFactorization(k,lr,reg_param,epochs)
print('Start Model Training')
P,Q,b_u,b_i = mf_model.fit(R_train)
print('Model Training Completed')

Start Model Training
Epoch : 0 , Loss : 297.235868 , Rooted Loss: 17.24
Epoch : 10 , Loss : 284.234457 , Rooted Loss: 16.86
Epoch : 20 , Loss : 276.417811 , Rooted Loss: 16.63
Epoch : 30 , Loss : 271.086198 , Rooted Loss: 16.46
Epoch : 40 , Loss : 267.121960 , Rooted Loss: 16.34
Model Training Completed


## 4. Prediction

* **Mf_model에 구현된 Predict() 메소드 참고**

- Prediction은 총 2방향으로 구현
    - exclude_unknowns=True
        - train set에서는 보지 못한 user_id,item_id에 대해서 모두 제외 (Cold Start Problem)하고 예측을 진행 (하지만 user,item이 동시에 같이 train/test dataset에 존재하는 Case가 존재X)
    - exclude_unknowns=False
        - train set에서 보지 못한 user_id,item_id에 대해서 총 4가지 경우를 통해 각기 다른 방법으로 처리.
            1. user_id,test_id 둘다 train에 없을때 : 0값을 반환 (결측치로 간주)
            2. user_id 만 train에 없을 때 : P(user latent matrix)의 값을 모두 1인 벡터로 설정, 내적에 Q값만 반영되도록 함. b_u (user bias) 항 또한 0으로 설정하여 효과를 없앰.
            3. item_id 만 train에 없을 때 : Q(item latent matrix)의 값을 모두 1인 벡터로 설정, 내적에 P값만 반영되도록 함. b_i (item bias) 항을 0으로 설정하여 효과를 없앰.
            4. test set의 item_id, user_id 모두 train에도 존재할때 : 원래 방식을 사용. 

- 문제점 : user_id에 상관없이, 대부분의 예측값이 거의 동일한 값 (3.098424)로 예측이 됨.

In [28]:
prediction,test = mf_model.predict(test,exclude_unknowns=False)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19388\1143839861.py:88: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  bias_i_product = int(bi_df.loc[val[1]])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19388\1143839861.py:92: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  bias_u_product = int(bu_df.loc[val[0]])


In [29]:
prediction

,user,item,rating
1,186,302,3.098424
7,253,465,3.098424
20,119,392,3.098424
21,167,486,3.098424
26,38,95,3.098424
...,...,...,...
99973,821,151,3.098424
99974,764,596,3.098424
99979,943,391,3.098424
99992,721,262,3.098424


In [ ]:
#Problem
prediction['rating'].value_counts()

rating
3.098424    19911
5.423558       54
5.629661       35
Name: count, dtype: int64

In [30]:
test

,user,item,rating
1,186,302,3.0
7,253,465,5.0
20,119,392,4.0
21,167,486,4.0
26,38,95,5.0
...,...,...,...
99973,821,151,4.0
99974,764,596,3.0
99979,943,391,2.0
99992,721,262,3.0


## 5. Evaluation Metrics

- Prediction metrics : RMSE
- Ranking metrics : Precision@k,recall@k,NDCG@k 구현

In [7]:
def precision_at_k(y_pred,y_true,k,threshold):
    prec_at_k = []
    for user_num in y_pred['user'].unique():
        top_test_items = y_true.loc[(y_true['user']==user_num)].sort_values('rating',ascending=False)
        top_test_rel_items = top_test_items.loc[top_test_items['rating'] > np.mean(top_test_items['rating'])]
        top_pred_items = y_pred.loc[(y_pred['user']==user_num)].sort_values('rating',ascending=False)[:k]      
        prec_at_k.append(len(set(top_test_rel_items['item']).intersection(set(top_pred_items['item']))) / k)
    return np.mean(prec_at_k)

def recall_at_k(y_pred,y_true,k,threshold): #Relevant Item 중에서 Hit (predict)한 item 갯수
    rec_at_k = []
    for user_num in y_pred['user'].unique():
        top_test_items = y_true.loc[(y_true['user']==user_num)].sort_values('rating',ascending=False)
        top_test_rel_items = top_test_items[top_test_items['rating'] > np.mean(top_test_items['rating'])]
        top_pred_items = y_pred.loc[(y_pred['user']==user_num)].sort_values('rating',ascending=False)[:k]
        top_pred_rel_items = top_pred_items[top_pred_items['rating'] > np.mean(top_test_items['rating'])]
        rec_at_k.append(len(set(top_test_rel_items['item']).intersection(set(top_pred_rel_items['item']))) / len(top_test_rel_items) if len(top_test_rel_items) >0 else 0)   #Intersection method requires set() data
    return np.mean(rec_at_k)

def ndcg_at_k(y_pred,y_true,k):
    ndcg_k = []
    for user_num in y_pred['user'].unique():
       top_pred_items = y_pred.loc[(y_pred['user']==user_num)].sort_values('rating',ascending=False)
       pred_sequence = top_pred_items['item'][:k].values

       test_items = y_true.loc[y_true['user']==user_num]
       ideal_rel_score = test_items.sort_values('rating',ascending=False)[:k]['rating'].values
       rel_score = test_items.set_index('item').reindex(pred_sequence)['rating'].values
       dcg_k = np.sum((np.pow(2,rel_score) -1) / np.log2(np.arange(2,len(rel_score)+2)))
       idcg_k = np.sum((np.pow(2,ideal_rel_score) -1) / np.log2(np.arange(2,len(ideal_rel_score)+2)))
       ndcg_k.append(dcg_k / idcg_k if idcg_k>0 else 0)
    
    return np.mean(ndcg_k)

In [33]:
rsme_par = root_mean_squared_error(prediction['rating'].values,test['rating'].values)
prec_at_k_par = precision_at_k(prediction,test,k=10,threshold=3)
rec_at_k_par = recall_at_k(prediction,test,k=10,threshold=3)
ndcg_at_k_par = ndcg_at_k(prediction,test,k=10)

print(f'RMSE : {rsme_par:.4f} , Precision@k : {prec_at_k_par:.4f} , Recall@k : {rec_at_k_par:.4f} , NDCG@k :{ndcg_at_k_par:.4f}')

RMSE : 1.2204 , Precision@k : 0.4437 , Recall@k : 0.1048 , NDCG@k :0.6894


# For Data_indicator = 0

- 앞선 경우와 적용하는 함수는 동일하지만, Cross Validation을 위해서 for문을 통한 반복문 작성

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
rmse = []
prec_at_k = []
rec_at_k = []
ndcg_value_k = []
n = 1

In [8]:
data_list = [['u1.base','u1.test'],
             ['u2.base','u2.test'],
             ['u3.base','u3.test'],
             ['u4.base','u4.test'],
             ['u5.base','u5.test']]

for b,t in data_list:
        print(f'\nFold {n} / Fold 5 Start')
        train,test,R_train = load_data(b,t,data_indicator=0)

        R_train = R_train.fillna(0)
        k = 10
        lr = 0.0001
        reg_param = 0.02
        epochs = 50

        mf_model  = MatrixFactorization(k,lr,reg_param,epochs)

        print('Start Model Training')
        mf_model.fit(R_train)
        print('\nModel Training Success')

        #Prediction
        prediction,test = mf_model.predict(test,exclude_unknowns=False)
    
        #Evaluation
        rsme_par = root_mean_squared_error(prediction['rating'].values,test['rating'].values)
        prec_at_k_par = precision_at_k(prediction,test,k=10,threshold=3)
        rec_at_k_par = recall_at_k(prediction,test,k=10,threshold=3)
        ndcg_at_k_par = ndcg_at_k(prediction,test,k=10)

        rmse.append(rsme_par)
        prec_at_k.append(prec_at_k_par)
        rec_at_k.append(rec_at_k_par)
        ndcg_value_k.append(ndcg_at_k_par)
        print(f'RMSE : {rsme_par:.4f} , Precision@k : {prec_at_k_par:.4f} , Recall@k : {rec_at_k_par:.4f} , NDCG@k :{ndcg_at_k_par:.4f}')

        print(f'Fold {n} / Fold 5 Completed')
        n=n+1 #Fold indicator
print('-'*8,'Model Training Finished','-'*8)
print(f'RMSE : {np.mean(rmse):.5f} , Precision@k : {np.mean(prec_at_k):.5f} , Recall@k : {np.mean(rec_at_k):.5f},\
           NDCG@k:{np.mean(ndcg_value_k):.5f}')


Fold 1 / Fold 5 Start
Data_indicator =0 Selected

Start Model Training
Epoch : 0 , Loss : 296.111600 , Rooted Loss: 17.21
Epoch : 10 , Loss : 283.550226 , Rooted Loss: 16.84
Epoch : 20 , Loss : 275.898047 , Rooted Loss: 16.61
Epoch : 30 , Loss : 270.641050 , Rooted Loss: 16.45
Epoch : 40 , Loss : 266.717139 , Rooted Loss: 16.33

Model Training Success
RMSE : 1.2493 , Precision@k : 0.5285 , Recall@k : 0.0747 , NDCG@k :0.6572
Fold 1 / Fold 5 Completed

Fold 2 / Fold 5 Start
Data_indicator =0 Selected

Start Model Training
Epoch : 0 , Loss : 294.940999 , Rooted Loss: 17.17
Epoch : 10 , Loss : 282.535479 , Rooted Loss: 16.81
Epoch : 20 , Loss : 274.969316 , Rooted Loss: 16.58
Epoch : 30 , Loss : 269.769701 , Rooted Loss: 16.42
Epoch : 40 , Loss : 265.883852 , Rooted Loss: 16.31

Model Training Success
RMSE : 1.2217 , Precision@k : 0.5020 , Recall@k : 0.0954 , NDCG@k :0.7012
Fold 2 / Fold 5 Completed

Fold 3 / Fold 5 Start
Data_indicator =0 Selected

Start Model Training
Epoch : 0 , Loss :